In [2]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
from sklearn.grid_search import (GridSearchCV, RandomizedSearchCV)
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier)
from sklearn import (metrics, cross_validation, linear_model, preprocessing)

In [5]:
def read_csv(day_trade):
    data_up = []
    data_down = []
    path = '/home/rory/SGX-OrderBook-Tick-Data-Trading-Strategy/Train_Test_Data/ML_data_2014'
    for j,i in enumerate(day_trade):
        for k in range(0,len(i),1):
            path_up = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'UP' + '.csv'
            path_down = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'DOWN' + '.csv'
            data_up.append(pd.read_csv(path_up))
            data_down.append(pd.read_csv(path_down))
            #print path_down
    return data_up,data_down

In [6]:
day_trade = [[2,3,6,7,8,10,13,14,15,16,17,20,21,22,23,24,27,28,29,30],\
             [7,10,11,12,13,17,18,19,21,24,25,26,27],\
             [3,4,5,6,7,10,11,13,14,17,18,19,20,24,25,26,27,31]]

In [7]:
data_2014_up, data_2014_down = read_csv(day_trade)

In [9]:
latest_sec = 60 * 30
pred_sec = 5

Accuracy_Mean_Day = []
Accuracy_Std_Day = []
Feature_Importance = []
Feature_Importance_Day_one = []
Feature_Importance_Day_two = []
Feature_Importance_Day_three = []
Feature_Importance_Day_four = []
Feature_Importance_Day_five = []
Predict_Values_Day = []
True_Values = []

for day in range(0,1,1):
    Accuracy = []
    Feature_Importance_one = []
    Feature_Importance_two = []
    Feature_Importance_three = []
    Feature_Importance_four = []
    Feature_Importance_five = []
    predict_values = []
    true_values = []
    CV_acc = []
    for i in range(0,9000-latest_sec-pred_sec,pred_sec):

        print '----------------------------Rolling Window Time = %s----------------------------'%(i/pred_sec)
        data_train = data_2014_up[day][i:i+latest_sec]
        X_train = data_train.drop(["0"],axis=1)
        y_train = data_train['0']

        data_test = data_2014_up[day][i+latest_sec:i+latest_sec+pred_sec]
        X_test = data_test.drop(["0"],axis=1)
        y_test = data_test['0']

        # hyper-parameter
        num_trees = [10]
        max_depth = [10]
        criterion = ['entropy']
        min_samples_leaf = [5]
        min_samples_split = [2]
        max_features = [None]

        # cv
        model_grid = {'max_features':max_features,'n_estimators':num_trees,'max_depth':max_depth,\
                      'min_samples_split':min_samples_split,'criterion':criterion,\
                      'min_samples_leaf':min_samples_leaf}
        
        model = RandomForestClassifier(random_state = 0)
        Grid = GridSearchCV(model, model_grid, cv = 5)
        %time Grid.fit(X_train,y_train) 
        print Grid.best_params_
        print 'CV Best Score = %s'%(Grid.best_score_)
        CV_acc.append(Grid.best_score_)
        # training & testing
        model.set_params(**Grid.best_params_)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        print 'Prediction latest 15 second = %s'%(predictions)
        predict_values.append(predictions.tolist())
        true_values.append(y_test.tolist())
        acc = metrics.accuracy_score(y_test, predictions)
        print 'Accuracy = %s'%(acc)
        Accuracy.append(acc)
        feature_imp = dict(zip([i for i in range(0,64,1)],model.feature_importances_))
        
        Top_five = sorted(feature_imp.items(),key = lambda x : x[1] , reverse=True)[0:5]
        print 'Top five importance features = %s'%(Top_five)
        
        Feature_Importance_one.append(Top_five[0][0])
        Feature_Importance_two.append(Top_five[1][0])
        Feature_Importance_three.append(Top_five[2][0])
        Feature_Importance_four.append(Top_five[3][0])
        Feature_Importance_five.append(Top_five[4][0])
        
    print 'Day = %s, mean = %s, std = %s'%(day,np.mean(Accuracy),np.std(Accuracy))
    
    Accuracy_Mean_Day.append(Accuracy)
    Accuracy_Std_Day.append(Accuracy)
    Feature_Importance_Day_one.append(Feature_Importance_one)
    Feature_Importance_Day_two.append(Feature_Importance_two)
    Feature_Importance_Day_three.append(Feature_Importance_three)
    Feature_Importance_Day_four.append(Feature_Importance_four)
    Feature_Importance_Day_five.append(Feature_Importance_five)
    Predict_Values_Day.append(predict_values)
    True_Values.append(true_values)
    

----------------------------Rolling Window Time = 0----------------------------
{'min_samples_leaf': 5, 'n_estimators': 10, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': None, 'max_depth': 10}
CV Best Score = 0.738333333333
Prediction latest 15 second = [ 1.  1.  1.  1.  1.]
Accuracy = 1.0
Top five importance features = [(26, 0.2756067374499116), (28, 0.19572412669985756), (30, 0.064353057612475117), (47, 0.061136974638440811), (31, 0.055952665806685717)]
----------------------------Rolling Window Time = 1----------------------------
{'min_samples_leaf': 5, 'n_estimators': 10, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': None, 'max_depth': 10}
CV Best Score = 0.728333333333
Prediction latest 15 second = [ 1.  1.  1.  1.  1.]
Accuracy = 1.0
Top five importance features = [(26, 0.31036863797482611), (31, 0.10395243530013572), (28, 0.095913518953415863), (29, 0.053916160535994719), (25, 0.046270784979101899)]
----------------------------Rolling Window 

KeyboardInterrupt: 

In [ ]:
Grid.best_score_

In [53]:
np.array(Accuracy_Mean_Day).mean(axis=1),np.array(Accuracy_Mean_Day).std(axis=1)

(array([ 0.89019608,  0.90588235,  0.93217139,  0.86361656,  0.90689906,
         0.87668845,  0.88002905,  0.897313  ,  0.913435  ,  0.88946986,
         0.88453159,  0.88816267,  0.87407407,  0.8681191 ,  0.91256354,
         0.90994916,  0.89455338,  0.90689906,  0.87102397,  0.90297749]),
 array([ 0.22719541,  0.22488979,  0.18429925,  0.26127519,  0.22016113,
         0.24124721,  0.24394006,  0.22490762,  0.20295825,  0.22966403,
         0.24737869,  0.22456885,  0.23892224,  0.24947841,  0.21676187,
         0.22171024,  0.22438353,  0.22503308,  0.2594902 ,  0.22975963]))

In [54]:
mean(np.array(Accuracy_Mean_Day).mean(axis=1))

0.89342774146695714

In [122]:
confusion_matrix(y_test,predictions)

array([[5]])

In [125]:
pd.crosstab(y_test,predictions,rownames = ['actual'],colnames=['preds'])

preds,1.0
actual,
1.0,5


In [44]:
latest_sec = 60 * 35
pred_sec = 15


Precision_Mean_Day = []
Precision_Std_Day = []
Feature_Importance = []
Feature_Importance_Day_one = []
Feature_Importance_Day_two = []
Feature_Importance_Day_three = []
Feature_Importance_Day_four = []
Feature_Importance_Day_five = []
Predict_Values_Day = []
True_Values_Day = []

for day in range(0,10,1):
    Precision = []
    Feature_Importance_one = []
    Feature_Importance_two = []
    Feature_Importance_three = []
    Feature_Importance_four = []
    Feature_Importance_five = []
    predict_values = []
    true_values = []
    for i in range(0,9000-latest_sec-pred_sec,pred_sec):
        print '----------------------------Rolling Window Time = %s----------------------------'%(i/pred_sec)
        data_train = data_2014_up[day][i:i+latest_min]
        X_train = data_train.drop(["0"],axis=1)
        y_train = data_train['0']

        data_test = data_2014_up[day][i+latest_sec:i+latest_sec+pred_sec]
        X_test = data_test.drop(["0"],axis=1)
        y_test = data_test['0']

        # hyper-parameter
        num_trees = [10]
        max_depth = [10]
        criterion = ['entropy']
        min_samples_leaf = [5]
        min_samples_split = [2]
        max_features = [None]

        # cv
        model_grid = {'max_features':max_features,'n_estimators':num_trees,'max_depth':max_depth,\
                      'min_samples_split':min_samples_split,'criterion':criterion,\
                      'min_samples_leaf':min_samples_leaf}
        
        model = RandomForestClassifier(random_state = 0)
        Grid = GridSearchCV(model, model_grid,cv = 5,scoring = 'precision')
        Grid.fit(X_train,y_train) 
        print Grid.best_params_
        print 'CV Best Score = %s'%(Grid.best_score_)

        # training & testing
        model.set_params(**Grid.best_params_)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        print 'Prediction latest 15 second = %s'%(predictions)
        predict_values.append(predictions.tolist())
        prec = metrics.precision_score(y_test, predictions)
        print 'Precision = %s'%(prec)
        Precision.append(prec)
        true_values.append(y_test.tolist())
        feature_imp = dict(zip([i for i in range(0,64,1)],model.feature_importances_))
        
        Top_five = sorted(feature_imp.items(),key = lambda x : x[1] , reverse=True)[0:5]
        print 'Top five importance features = %s'%(Top_five)
        
        Feature_Importance_one.append(Top_five[0][0])
        Feature_Importance_two.append(Top_five[1][0])
        Feature_Importance_three.append(Top_five[2][0])
        Feature_Importance_four.append(Top_five[3][0])
        Feature_Importance_five.append(Top_five[4][0])
        
    print 'Day = %s, mean = %s, std = %s'%(day,np.mean(Precision),np.std(Precision))
    Accuracy_Mean_Day.append(np.mean(Precision))
    Accuracy_Std_Day.append(np.std(Precision))
    Feature_Importance_Day_one.append(Feature_Importance_one)
    Feature_Importance_Day_two.append(Feature_Importance_two)
    Feature_Importance_Day_three.append(Feature_Importance_three)
    Feature_Importance_Day_four.append(Feature_Importance_four)
    Feature_Importance_Day_five.append(Feature_Importance_five)
    Predict_Values_Day.append(predict_values)
    

----------------------------Rolling Window Time = 0----------------------------
{'min_samples_leaf': 5, 'n_estimators': 10, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': None, 'max_depth': 10}
CV Best Score = 0.866001354116
Prediction latest 15 second = [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
Precision = 1.0
Top five importance features = [(26, 0.28135318632866974), (28, 0.14809374175586734), (25, 0.099954422451823371), (31, 0.090794981214531126), (36, 0.03264190338758223)]
----------------------------Rolling Window Time = 1----------------------------
{'min_samples_leaf': 5, 'n_estimators': 10, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': None, 'max_depth': 10}
CV Best Score = 0.871295801581
Prediction latest 15 second = [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
Precision = 1.0
Top five importance features = [(26, 0.41542656057596927), (28, 0.069637651860375668), (48, 0.053510542753711143), (25, 0.052083436

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'min_samples_leaf': 5, 'n_estimators': 10, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': None, 'max_depth': 10}
CV Best Score = 0.897355919572
Prediction latest 15 second = [ 1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  1.  0.  1.  1.]
Precision = 1.0
Top five importance features = [(24, 0.52395073860467578), (23, 0.130412812450692), (25, 0.053650538788705404), (29, 0.04028797674703477), (5, 0.032369503420124138)]
----------------------------Rolling Window Time = 17----------------------------
{'min_samples_leaf': 5, 'n_estimators': 10, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': None, 'max_depth': 10}
CV Best Score = 0.89999677829
Prediction latest 15 second = [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
Precision = 1.0
Top five importance features = [(24, 0.38967246317926124), (23, 0.24003634424151077), (28, 0.065529259063178913), (18, 0.041640400747271773), (25, 0.031661927109245161)]
----------------------------Rolling Window T

KeyboardInterrupt: 

In [46]:
predict_values

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [47]:
true_values

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.